In [3]:
import sys
import os
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import importlib

sys.path.append(os.path.abspath(".."))

import src.models.cb.cb_cv_trainer as cv
import src.utils.optuna_visualizer as opv
import src.utils.telegram as te

In [4]:
# Load data
env_path = Path.cwd().parent / ".env"
load_dotenv(dotenv_path=env_path)
url = os.environ.get("OPTUNA_STORAGE_URL")

tr_df1 = pd.read_parquet("../artifacts/features/base/tr_df1.parquet")
test_df1 = pd.read_parquet("../artifacts/features/base/test_df1.parquet")

In [ ]:
# Check tuning results
study_name = "cb_v1"

study = opv.OptunaVisualizer(study_name, url)
study.visualize_optimization()
study.print_trials_table()

In [ ]:
# Create OOF and test predictions
importlib.reload(cv)
params = {
    "learning_rate": 0.02,
    "depth": 12,
    "min_data_in_leaf": 29.394632044566997,
    "l2_leaf_reg": 19.697768782973583,
    "bagging_temperature": 0.7908265179488759,
    "random_strength": 68,
    "border_count": 206,
    "task_type": "CPU"
}

trainer = cv.CBCVTrainer(
    params=params,
    early_stopping_rounds=500)
oof, test_preds = trainer.fit(tr_df1, test_df1)
np.save("../artifacts/preds/base/oof_single_1.npy", oof)
np.save("../artifacts/preds/base/test_single_1.npy", test_preds)

In [21]:
te.send_telegram_message("CB Training complete!")

✅ Message sent successfully.


In [ ]:
# Full training
params = {
    "learning_rate": 0.02,
    "depth": 12,
    "min_data_in_leaf": 29.394632044566997,
    "l2_leaf_reg": 19.697768782973583,
    "bagging_temperature": 0.7908265179488759,
    "random_strength": 68,
    "border_count": 206,
    "task_type": "CPU",
    "iterations": int(577.4 * 1.25)
}
trainer = cv.CBCVTrainer(params=params)
trainer.full_train(tr_df1, test_df1, 1)